# This Class is used to handle the images the user uploads

The images will be cropped and rotated to prepare for embedding and querying the vector database.

In [1]:
import cv2
import numpy as np

def detect_and_crop_cards(image_path, min_area=5000):
    """
    Detect and crop cards from the image.
    Automatically rotate them to be vertical without altering colors.
    Args:
    - image_path: Path to the image containing the cards.
    - min_area: Minimum area to consider a contour as a card.
    
    Returns:
    - cropped_cards: List of cropped and rotated card images.
    """
    # Load the image without any color alterations
    image = cv2.imread(image_path)
    
    # Convert to grayscale for edge detection (no color alteration)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Edge detection to find card contours
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    cropped_cards = []
    
    for contour in contours:
        # Filter by area to ignore small contours
        area = cv2.contourArea(contour)
        if area < min_area:
            continue
        
        # Get the minimum area rectangle for the contour
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        
        # Crop the card from the image using the bounding box
        width = int(rect[1][0])
        height = int(rect[1][1])
        
        # Get the rotation matrix
        src_pts = box.astype("float32")
        dst_pts = np.array([[0, height-1],
                            [0, 0],
                            [width-1, 0],
                            [width-1, height-1]], dtype="float32")
        
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
        warped = cv2.warpPerspective(image, M, (width, height))
        
        # Ensure the card is rotated to be vertical
        if height > width:
            warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)
        
        cropped_cards.append(warped)
    
    return cropped_cards

In [ ]:
# Example usage:
image_paths = ['Datasets/mtg_test_images/shuko_mountain_carpet.jpg', 'Datasets/mtg_test_images/shuko_single_held_in_hand.jpg', 
                'Datasets/mtg_test_images/shuko_single_rotated_countertop.jpg', 'Datasets/mtg_test_images/shuko_single_slightly_rotated_countertop.jpg', 
                'Datasets/mtg_test_images/shuko_single_slightly_rotated_tablecloth.jpg', 'Datasets/mtg_test_images/shuko.jpg']

cards = [detect_and_crop_cards(image_path) for image_path in image_paths]

# Now you can preprocess and embed these cropped cards
for idx, card in enumerate(cards):
    # Save the card
    cv2.imwrite(f"cropped_card_{idx}.jpg", card)
